# Creación de paquetes DNS

### Descripción: 
Con este código se va a intentar crear diferentes tipos de paquetes DNS, mediante IA generativa. Se pretende pasar una descripción de estos paquetes al modelo y que este sea capaz de crear todos los campos necesarios para replicar un paquete con esas mismas especificaciones.

In [8]:
import random
import re
import pickle
import os

import openai
from openai import OpenAI
os.environ['OPENAI_API_KEY'] = 'Fill'
client = OpenAI()

from scapy.all import *
from scapy.utils import RawPcapReader, wrpcap
import scapy.all as scapy

from scapy.layers.l2 import Ether
from scapy.layers.inet import IP
from scapy.all import DNS

openai.__version__

C:\Users\adayd\anaconda3\envs\PAC-GPT\Lib\site-packages\scapy\layers\ipsec.py:469: CryptographyDeprecationWarning: Blowfish has been deprecated
  cipher=algorithms.Blowfish,
C:\Users\adayd\anaconda3\envs\PAC-GPT\Lib\site-packages\scapy\layers\ipsec.py:483: CryptographyDeprecationWarning: CAST5 has been deprecated
  cipher=algorithms.CAST5,


'1.10.0'

## Generación de la descipción de un paquete ICMP

<center><img src="https://miro.medium.com/v2/resize:fit:1400/1*EBKCNBODkcQMuvtXq9Vhog.png">
    
<center> Teniendo estos mensajes enviados la siguiente cabecera.

<center><img src="https://www.google.com/url?sa=i&url=http%3A%2F%2Fbastionado.blogspot.com%2F2013%2F02%2Fconsulta-dns-con-snort.html&psig=AOvVaw1cFLv_RkEKpsD_AUT71cnj&ust=1711802500271000&source=images&cd=vfe&opi=89978449&ved=0CBIQjRxqFwoTCPjIi7W_mYUDFQAAAAAdAAAAABAJ">

In [43]:
system_message = "In the next lines, I'll provide you with some packet descriptions for different types of DNS packets and how to construct each one of them using scapy. Use each one of this as a base to guide you while creating packets if they are the same type, but DONT create ALL these types of packets for every description, evaluate to wich type does the summary belong, and generate JUST that packet command.\n\n"

# Standard DNS Query
system_message += "This is a standard DNS query packet summary:\n"
system_message += "192.168.2.102 192.168.2.1 DNS 86 Standard query 0xd7fa PTR 80.144.99.193.in-addr.arpa\n"
system_message += "The command to generate this packet would be:\n"
system_message += 'IP(src="192.168.2.102", dst="192.168.2.1", proto=17)/UDP(sport=random.randint(4097, 65530), dport=53)/DNS(id=0xd7fa, qr=0, rd=1, opcode=0, qdcount=1, ancount=0, nscount=0, arcount=0, qd=DNSQR(qname="80.144.99.193.in-addr.arpa", qtype="PTR", qclass="IN"))\n\n'

#Standard DNS Response
system_message += "This is a standard DNS query response packet summary:\n"
system_message += "9.9.9.9 193.24.227.230 DNS 127 Standard query response 0x0aba A many-rrs.weberdns.de A 192.0.2.43 A 192.0.2.42 OPT\n"
system_message += "This is the python code to generate the packet:\n"
system_message += 'IP(src="9.9.9.9", dst="193.24.227.230", proto=17)/UDP(sport=53, dport=random.randint(4097, 65530))/DNS(id=0x0aba, qr=1, opcode=0, ra=1, rcode=0, qdcount=1, ancount=2, nscount=0, arcount=1, qd=DNSQR(qname="many-rrs.weberdns.de", qtype="A", qclass="IN"), an=DNSRR(rrname="many-rrs.weberdns.de", type="A", rclass="IN", ttl=255, rdata="192.0.2.43")/DNSRR(rrname="many-rrs.weberdns.de", type="A", rclass="IN", ttl=255, rdata="192.0.2.42"), ar=DNSRROPT(rrname=".", type=41, rclass=4096, z=0x0000 rdlen=None, rdata=""))\n\n'

#query += 'A value for varible "var_1" must be defined randomly in every new packet. You must generate a random port number for every new packet and substitute it for the variable. for example, if you randomly generate number 395, in a DNS query, the UDP part must me UDP(sport=395, dport=53). This is EXTREMELY important.\n\n'

system_message += "Please, be especially carefull with the variables and flags, take your time to consider if the summary means recursiveness in the query, if the response is authoritative, etc\n"
system_message += "The variable 'qr' depends on the type of the packet, queries qr=0 and responses have opcode=1\n\n"

system_message += "Take into consideration especificacitions of each query like the type and the class\n"
system_message += "The 'type' variable of the query specificies the type of the query, it can be: A(host address)->type=1, CNAME->type=5, PTR(domain name pointer)->type=12, MX(Mail exchange)->type=15, and much more.\n"
system_message += "The 'class' variable can be: 1= Internet(IN), 255= QCLASS *ANY, etc\n\n"

system_message += "While creating de commands, take your time to process the summaries. Dont create commands with obvious errors like repeated variables.\n"
system_message += "Dont create comands with both DNSQR (DNS Question Record) and  DNSRR (DNS Response Record) if the packet is a query, this is an obvious error as a query cant have its own response\n"

system_message += "The ancount variable gives us the number of an answers in the responses, it must be coherent with this part, ESPECIALLY IN THE QUERY RESPONSES, the same goes for the other (qdcount, nscount, arcount)\n"
system_message += "If you need to put various qd, an, ns, ar in the packet, they are added as follows, for example, if you want to add 3 ar : ar=DNSRR(...)/DNSRR(..)/DNSRR(...). This is really important, and the same structure (separated by / ) aplies to an, qd, ns and ar \n"
system_message += "Use DNSRRSOA(..) to create SOA responses, DNSRROPT(..) for OPT responses, etc\n\n"

system_message += "Take a few seconds (5s) to think about your answer and consider all requirements previosuly established, don't come up with an answer until all of them are satisfied\n\n"


In [45]:
print(system_message)

In the next lines, I'll provide you with some packet descriptions for different types of DNS packets and how to construct each one of them using scapy. Use each one of this as a base to guide you while creating packets if they are the same type, but DONT create ALL these types of packets for every description, evaluate to wich type does the summary belong, and generate JUST that packet command.

This is a standard DNS query packet summary:
192.168.2.102 192.168.2.1 DNS 86 Standard query 0xd7fa PTR 80.144.99.193.in-addr.arpa
The command to generate this packet would be:
IP(src="192.168.2.102", dst="192.168.2.1", proto=17)/UDP(sport=random.randint(4097, 65530), dport=53)/DNS(id=0xd7fa, qr=0, rd=1, opcode=0, qdcount=1, ancount=0, nscount=0, arcount=0, qd=DNSQR(qname="80.144.99.193.in-addr.arpa", qtype="PTR", qclass="IN"))

This is a standard DNS query response packet summary:
9.9.9.9 193.24.227.230 DNS 127 Standard query response 0x0aba A many-rrs.weberdns.de A 192.0.2.43 A 192.0.2.42 OPT

## Carga de posibles descripciones

In [16]:
with open("../../data/text/DNS_summaries.txt","r") as f:
    packets_summary = f.read().splitlines()
len(packets_summary)

526

### Carga de posibles conversaciones anteriores para concatenarlas

In [20]:
class response():
    "Stores name and place pairs"
    def __init__(self, name, place):
        self.prompt_summary = name
        self.completion = place

In [61]:
try:    
    responses = pickle.load( open("../../data/Packets/DNS/pickle/DNS_packet_pairs_Aday.pkl", "rb" ))
except:
    responses = []

## Generación de comandos de paquetes DNS

In [50]:
summaries_per_request = 10 # por que normalmente solo querremos generar una conversación

In [52]:
for i in range(10):
    
    summaries = random.choices(packets_summary,k=summaries_per_request)
    
    print("...........................................")
    print( " GENERATING SCAPY COMMANDS FOR THE FOLLOWING PACKETS (iter " +str(i+1) +")")
    

    text_sum = ""
    
    query_content = "\nBelow, I'll give you the descriptions of the packets you have to generate, the previous where just examples\n\n"
    
    for summary in summaries:
        query_content += "This is a packet summary:\n"
        query_content += summary + "\n"
        text_sum += summary + "\n"
        
    query_content += "\nGenerate python code for creating JUST the packets of the summaries given with scapy framework and put them all in a list named pkt_list. DO NOT create commands for other types of DNS rather than the ones belonging to the packets wanted to generate.Dont give back any comments or imports, just bare code and commands. Dont start the resposnses wiht ```python This is a flagrant error.\n"
    
    print(query_content)
    
    messages=[{"role": "system", "content": system_message},
        {"role": "user", "content": query_content}]
    
    #Esta es la petición que le mandas al modelo de IA
    completion = client.chat.completions.create(
        model="gpt-4o", 
        messages=messages,
        max_tokens=2600,
        temperature=0.1)
    
    print("...Generated!")
    
    # Guardamos tanto el resumen de los paquetes que solicitamos y lo que devuelve el modelo para estos paquetes
    responses.append(response(text_sum, completion))
    
    exec(responses[-1].completion.choices[0].message.content)
    
    with open("../../data/Packets/DNS/pcap/DNSgenerated_gpt4o.pcap", "ba+") as f:
        wrpcap(f, pkt_list, append=True)
    
    # Guardamos tanto la petición como la respuesta en un archivo
    pickle.dump(responses, open( "../../data/Packets/DNS/pickle/DNS_packet_pairs_Aday.pkl", "wb" ) )
    
    print("Number of completitions done: " + str(len(responses)))

...........................................
 GENERATING SCAPY COMMANDS FOR THE FOLLOWING PACKETS (iter 1)

Below, I'll give you the descriptions of the packets you have to generate, the previous where just examples

This is a packet summary:
130.231.202.234 â†’ 130.231.240.70 DNS 126 Standard query 0x18e0 A redirect.prod.experiment.routing.cloudfront.aws.a2z.com OPT
This is a packet summary:
130.231.202.234 â†’ 130.231.240.70 DNS 76 Standard query 0x4bcd A sstats.fdown.net
This is a packet summary:
130.231.240.70 â†’ 130.231.202.234 DNS 148 Standard query response 0x8a50 No such name A stats.fdown.net.oulu.fi SOA ousrvr.oulu.fi OPT
This is a packet summary:
130.231.202.234 â†’ 130.231.240.70 DNS 75 Standard query 0xe044 A usage.fdown.net
This is a packet summary:
130.231.240.70 â†’ 130.231.202.234 DNS 378 Standard query response 0xc306 AAAA ipv6check-http.steamserver.net CNAME http.ipv6check.akadns.net CNAME eu.http.ipv6check.akadns.net AAAA 2a01:bc80:4:101::35 AAAA 2a01:bc80:4:101::38

In [181]:
# Este es el resumen de lo que está en el struct de responses para la última petición
print(responses[-1].prompt_summary)
print(responses[-1].completion.choices[0].text)

130.231.202.234 â†’ 130.231.240.70 DNS 97 Standard query 0x1043 A encrypted-tbn0.gstatic.com OPT
130.231.240.70 â†’ 130.231.202.234 DNS 148 Standard query response 0x23f8 No such name A usage.fdown.net.oulu.fi SOA ousrvr.oulu.fi OPT
130.231.202.234 â†’ 130.231.240.70 DNS 94 Standard query 0x8939 A api-partner.spotify.com OPT
130.231.240.70 â†’ 130.231.202.234 DNS 201 Standard query response 0x3407 A platform.openai.com CNAME platform.api.openai.com CNAME platform.openai.com.cdn.cloudflare.net A 104.18.1.104 A 104.18.0.104 OPT
130.231.202.234 â†’ 130.231.240.70 DNS 105 Standard query 0x3c1c A audio-ak-spotify-com.akamaized.net OPT


pkt_list = []

# Packet 1
pkt1 = IP(src="130.231.202.234", dst="130.231.240.70", proto=17)/UDP(sport=12345, dport=53)/DNS(id=0x1043, qr=0, rd=1, opcode=0, qdcount=1, ancount=0, nscount=0, arcount=1, qd=DNSQR(qname="encrypted-tbn0.gstatic.com", qtype="A", qclass="IN"), ar=DNSRROPT(rrname=".", type=41, rclass=4096, z=0x0000, rdlen=None, rdata=""))
pkt_list.app

### Generación de paquetes

In [183]:
# Ejecutamos los comandos que devuelve el modelo para generar los paquetes
exec(responses[-1].completion.choices[0].text)
print(pkt_list)

[<IP  frag=0 proto=udp src=130.231.202.234 dst=130.231.240.70 |<UDP  sport=12345 dport=domain |<DNS  id=4163 qr=0 opcode=QUERY rd=1 qdcount=1 ancount=0 nscount=0 arcount=1 qd=<DNSQR  qname='encrypted-tbn0.gstatic.com' qtype=A qclass=IN |> ar=<DNSRROPT  rrname='.' type=OPT rclass=4096 z=0 rdlen=None rdata=[''] |> |>>>, <IP  frag=0 proto=udp src=130.231.240.70 dst=130.231.202.234 |<UDP  sport=domain dport=12345 |<DNS  id=9208 qr=1 opcode=QUERY ra=1 rcode=name-error qdcount=1 ancount=1 nscount=0 arcount=1 qd=<DNSQR  qname='usage.fdown.net.oulu.fi' qtype=A qclass=IN |> an=<DNSRR  rrname='usage.fdown.net.oulu.fi' type=SOA rclass=IN ttl=255 rdata='ousrvr.oulu.fi' |<DNSRROPT  rrname='.' type=OPT rclass=4096 z=0 rdlen=None rdata=[''] |>> |>>>, <IP  frag=0 proto=udp src=130.231.202.234 dst=130.231.240.70 |<UDP  sport=23456 dport=domain |<DNS  id=35129 qr=0 opcode=QUERY rd=1 qdcount=1 ancount=0 nscount=0 arcount=1 qd=<DNSQR  qname='api-partner.spotify.com' qtype=A qclass=IN |> ar=<DNSRROPT  rrna

In [184]:
print(responses[-1].prompt_summary)
IP(pkt_list[2]).show()

130.231.202.234 â†’ 130.231.240.70 DNS 97 Standard query 0x1043 A encrypted-tbn0.gstatic.com OPT
130.231.240.70 â†’ 130.231.202.234 DNS 148 Standard query response 0x23f8 No such name A usage.fdown.net.oulu.fi SOA ousrvr.oulu.fi OPT
130.231.202.234 â†’ 130.231.240.70 DNS 94 Standard query 0x8939 A api-partner.spotify.com OPT
130.231.240.70 â†’ 130.231.202.234 DNS 201 Standard query response 0x3407 A platform.openai.com CNAME platform.api.openai.com CNAME platform.openai.com.cdn.cloudflare.net A 104.18.1.104 A 104.18.0.104 OPT
130.231.202.234 â†’ 130.231.240.70 DNS 105 Standard query 0x3c1c A audio-ak-spotify-com.akamaized.net OPT

###[ IP ]### 
  version   = 4
  ihl       = 5
  tos       = 0x0
  len       = 80
  id        = 1
  flags     = 
  frag      = 0
  ttl       = 64
  proto     = udp
  chksum    = 0xb99c
  src       = 130.231.202.234
  dst       = 130.231.240.70
  \options   \
###[ UDP ]### 
     sport     = 23456
     dport     = domain
     len       = 60
     chksum    = 0x9c

In [185]:
#Enviarlos para probar si se envian correctamente
send(pkt_list)


Sent 5 packets.


### Introducción en un pcap

In [187]:
# Guardamos los paquetes generados en un pcap
with open("../../data/Packets/DNS/pcap/DNSgenerated_3_5_turbo-instruct_conv.pcap", "ba+") as f:
    wrpcap(f, pkt_list, append=True)